# Jupyter Example
## LAMMPS: body/in.pour3d
Reference: https://github.com/lammps/lammps/blob/master/examples/body/in.pour3d  
Required files: molecule.cube, molecule.point3d, molecule.rod3d, molecule.tetra  
Description: pouring 3d rounded polyhedron bodies

In [1]:
from __future__ import print_function
import random, math
from lammps import IPyLammps

In [2]:
L = IPyLammps()

L.variable("steps index 6000")
L.units("lj")
L.boundary("p p fm")
L.comm_modify("vel yes")
L.atom_style("body rounded/polyhedron 1 8")
L.atom_modify("map array")
L.region("reg block 0 50 0 50 0 50 units box")
L.create_box("4 reg")

LAMMPS output is captured by PyLammps wrapper


['Created orthogonal box = (0 0 0) to (50 50 50)',
 '  1 by 1 by 1 MPI processor grid']

In [3]:
L.variable("cut_inner equal 0.5")
L.variable("k_n equal 100")
L.variable("k_na equal 5")
L.variable("c_n equal 20")
L.variable("c_t equal 5")
L.variable("mu equal 0")
L.variable("A_ua equal 1")
L.pair_style("body/rounded/polyhedron ${c_n} ${c_t} ${mu} ${A_ua} ${cut_inner}")
L.pair_coeff("* * ${k_n} ${k_na}")

In [4]:
L.neighbor("0.5 bin")
L.neigh_modify("every 1 delay 0 check yes")

L.fix("1 all nve/body")
L.fix("2 all gravity 1.0 spherical 0.0 -180.0")

L.molecule("object molecule.cube molecule.tetra toff 1 molecule.rod3d toff 2 molecule.point3d toff 3")
L.region("slab block 5 45 5 45 25 35 units box")

L.fix("ins all pour 500 0 4767548 vol 0.4 10 region slab mol object molfrac 0.25 0.25 0.25 0.25")
L.fix("4 all wall/body/polyhedron 2000 50 50 zplane 0.0 NULL")

In [5]:
L.compute("1 all body/local type 1 2 3")
L.dump("1 all local 1000 dump.polyhedron index c_1[1] c_1[2] c_1[3] c_1[4]")
L.dump("10 all custom 1000 tmp.dump id type x y z radius")

#L.dump("2 all image 500 image.*.jpg type type zoom 1.5 adiam 1.5 body type 0 0 view 75 15")
#L.dump_modify("2 pad 6")

L.dump("2 all movie 1000 mov.mp4 type type zoom 1.2 adiam 1.5 body type 0 0 view 75 15")
L.dump_modify("2 pad 6")

In [6]:
L.thermo_style("custom step atoms ke pe etotal press")
L.thermo("1000")
L.timestep("0.001")
L.run("${steps}")

['Neighbor list info ...',
 '  update every 1 steps, delay 0 steps, check yes',
 '  max neighbors/atom: 2000, page size: 100000',
 '  master list distance cutoff = 5',
 '  ghost atom cutoff = 5',
 '  binsize = 2.5, bins = 20 20 20',
 '  1 neighbor lists, perpetual/occasional/extra = 1 0 0',
 '  (1) pair body/rounded/polyhedron, perpetual',
 '      attributes: half, newton on',
 '      pair build: half/bin/atomonly/newton',
 '      stencil: half/bin/3d/newton',
 '      bin: standard',
 'Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 0',
 '  Time step     : 0.001',
 'Per MPI rank memory allocation (min/avg/max) = 0.5065 | 0.5065 | 0.5065 Mbytes',
 'Step Atoms KinEng PotEng TotEng Press ',
 '       0        0           -0            0            0            0 ',
 'WARNING: Less insertions than requested (../fix_pour.cpp:679)',
 '    1000      175           -0   0.02073543   0.02073543 -6.062863e-05 ',
 'WARNING: Less insertions than requested (../fix_pour.cpp:67

In [7]:
L.undump(2)
L.video("mov.mp4")